In [4]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import VideoMode
import cv2
import numpy as np
import threading
import queue
import time
import gc

# ===== CONFIGURATION - 1080p =====
WIDTH, HEIGHT = 1920, 1080
PROCESS_SCALE = 0.25  # Process at 480x270 (aggressive downscale for 1080p)
BUFFER_SIZE = 2
TARGET_FPS = 30
MOTION_THRESHOLD = 25

# ===== CONFIGURE FPGA & HDMI =====
print("Initializing FPGA for 1080p...")
base = BaseOverlay("base.bit")
hdmi_out = base.video.hdmi_out

mode = VideoMode(WIDTH, HEIGHT, 24)
hdmi_out.configure(mode)
hdmi_out.start()
print(f"HDMI ready: {WIDTH}x{HEIGHT}")

# ===== CONFIGURE CAMERA =====
print("Initializing camera for 1080p...")
cap = cv2.VideoCapture(0, cv2.CAP_V4L2)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'))
cap.set(cv2.CAP_PROP_FRAME_WIDTH, WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, HEIGHT)
cap.set(cv2.CAP_PROP_FPS, TARGET_FPS)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

actual_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
actual_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"Camera ready: {actual_w}x{actual_h}")

if actual_w != WIDTH or actual_h != HEIGHT:
    print(f"WARNING: Camera doesn't support 1080p natively")

# ===== STREAMING COMPONENTS =====
frame_queue = queue.Queue(maxsize=BUFFER_SIZE)
result_queue = queue.Queue(maxsize=BUFFER_SIZE)

class StreamState:
    def __init__(self):
        self.running = True
        self.fps_capture = 0.0
        self.fps_process = 0.0
        self.fps_display = 0.0
        self.motion_detected = False
        self.motion_count = 0
        self.dropped_frames = 0
        
state = StreamState()

# ===== THREAD 1: CAPTURE =====
def capture_thread():
    frame_count = 0
    start_time = time.perf_counter()
    last_reset = start_time
    
    while state.running:
        ret, frame = cap.read()
        if not ret:
            time.sleep(0.001)
            continue
        
        # Force 1080p if camera returns different size
        if frame.shape[:2] != (HEIGHT, WIDTH):
            frame = cv2.resize(frame, (WIDTH, HEIGHT))
        
        try:
            frame_queue.put(frame, block=False)
            frame_count += 1
        except queue.Full:
            state.dropped_frames += 1
        
        # Calculate FPS every 30 frames with rolling window
        if frame_count % 30 == 0:
            now = time.perf_counter()
            elapsed = now - last_reset
            state.fps_capture = 30 / elapsed  # FPS of last 30 frames
            last_reset = now

# ===== THREAD 2: MOTION DETECTION =====
def detection_thread():
    prev_gray = None
    process_w = int(WIDTH * PROCESS_SCALE)
    process_h = int(HEIGHT * PROCESS_SCALE)
    
    frame_count = 0
    start_time = time.perf_counter()
    last_reset = start_time
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    
    while state.running:
        try:
            frame = frame_queue.get(timeout=0.1)
        except queue.Empty:
            continue
        
        # Aggressive downscale for 1080p
        small = cv2.resize(frame, (process_w, process_h))
        gray = cv2.cvtColor(small, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (15, 15), 0)  # Reduced kernel for speed
        
        motion_contours = []
        
        if prev_gray is not None:
            diff = cv2.absdiff(prev_gray, gray)
            _, thresh = cv2.threshold(diff, MOTION_THRESHOLD, 255, cv2.THRESH_BINARY)
            
            thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
            thresh = cv2.dilate(thresh, kernel, iterations=1)
            
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            scale_factor = 1.0 / PROCESS_SCALE
            for c in contours:
                if cv2.contourArea(c) > 80:
                    x, y, w, h = cv2.boundingRect(c)
                    motion_contours.append((
                        int(x * scale_factor),
                        int(y * scale_factor),
                        int(w * scale_factor),
                        int(h * scale_factor)
                    ))
            
            state.motion_detected = len(motion_contours) > 0
            state.motion_count = len(motion_contours)
        
        prev_gray = gray
        
        try:
            result_queue.put((frame, motion_contours), block=False)
            frame_count += 1
        except queue.Full:
            pass
        
        # Calculate FPS every 30 frames with rolling window
        if frame_count % 30 == 0:
            now = time.perf_counter()
            elapsed = now - last_reset
            state.fps_process = 30 / elapsed  # FPS of last 30 frames
            last_reset = now

# ===== THREAD 3: DISPLAY =====
def display_thread():
    frame_count = 0
    prev_time = time.perf_counter()
    fps_smooth = 0.0
    
    hdmi_buffer = hdmi_out.newframe()
    
    while state.running:
        try:
            frame, contours = result_queue.get(timeout=0.1)
        except queue.Empty:
            continue
        
        # Measure total time between frames (includes queue wait)
        current_time = time.perf_counter()
        time_diff = current_time - prev_time
        prev_time = current_time
        
        # Draw motion boxes (thinner lines for 1080p)
        for (x, y, w, h) in contours:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 3)
        
        # Status overlay (larger text for 1080p)
        status_color = (0, 255, 0) if state.motion_detected else (100, 100, 100)
        status_text = "MOTION" if state.motion_detected else "NO MOTION"
        cv2.putText(frame, status_text, (30, 70), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1.8, status_color, 3)
        
        # Performance stats
        cv2.putText(frame, f"Capture: {state.fps_capture:.1f} FPS", (30, 140), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 0), 2)
        cv2.putText(frame, f"Process: {state.fps_process:.1f} FPS", (30, 190), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 0), 2)
        cv2.putText(frame, f"Display: {fps_smooth:.1f} FPS", (30, 240), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 2)
        
        cv2.putText(frame, f"Motion Regions: {state.motion_count}", (30, 290), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)
        
        if state.dropped_frames > 0:
            cv2.putText(frame, f"Dropped: {state.dropped_frames}", (30, 340), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        
        # Resolution indicator
        cv2.putText(frame, "1920x1080", (WIDTH - 250, 70), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 255), 2)
        
        # Convert and write to HDMI
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        hdmi_buffer[:] = frame_rgb
        hdmi_out.writeframe(hdmi_buffer)
        
        # Calculate actual FPS (including all delays)
        if time_diff > 0:
            instant_fps = 1.0 / time_diff
            # Exponential moving average for smooth display
            fps_smooth = 0.9 * fps_smooth + 0.1 * instant_fps
        
        frame_count += 1
        state.fps_display = fps_smooth

# ===== MAIN EXECUTION =====
print("=" * 60)
print("1080p MOTION DETECTION PIPELINE")
print(f"Camera Input:  {WIDTH}x{HEIGHT}")
print(f"Processing:    {int(WIDTH*PROCESS_SCALE)}x{int(HEIGHT*PROCESS_SCALE)}")
print(f"HDMI Output:   {WIDTH}x{HEIGHT}")
print(f"Pixel Count:   {WIDTH*HEIGHT:,} (2.25x more than 720p)")
print("=" * 60)
print("\nPress Ctrl+C to stop\n")

try:
    threads = [
        threading.Thread(target=capture_thread, daemon=True),
        threading.Thread(target=detection_thread, daemon=True),
        threading.Thread(target=display_thread, daemon=True)
    ]
    
    for t in threads:
        t.start()
    
    print("Pipeline started")
    
    while True:
        time.sleep(5)
        print(f"[1080p] Cap:{state.fps_capture:.1f} | "
              f"Proc:{state.fps_process:.1f} | "
              f"Disp:{state.fps_display:.1f} | "
              f"Motion:{state.motion_count} | "
              f"Drop:{state.dropped_frames}")

except KeyboardInterrupt:
    print("\nStopping...")
    state.running = False
    
    for t in threads:
        t.join(timeout=1.0)
    
finally:
    print("Cleanup...")
    cap.release()
    hdmi_out.stop()
    del hdmi_out
    gc.collect()
    
    print("\n" + "=" * 60)
    print("FINAL STATISTICS (1080p):")
    print(f"Capture FPS:  {state.fps_capture:.2f}")
    print(f"Process FPS:  {state.fps_process:.2f}")
    print(f"Display FPS:  {state.fps_display:.2f}")
    print(f"Dropped:      {state.dropped_frames} frames")
    print("=" * 60)
    print("Done")

Initializing FPGA...
HDMI ready: 1280x720
Initializing camera...
Camera ready: 1280x720
LIGHTWEIGHT MOTION DETECTION PIPELINE
Camera Input:  1280x720
Processing:    384x216
HDMI Output:   1280x720
Buffer Depth:  2 frames
Algorithm:     Frame Differencing (Low Power)

Press Ctrl+C to stop

Pipeline started
[720p] Cap:6.9 | Proc:6.5 | Disp:6.9 | Motion:0 | Drop:0
[720p] Cap:7.2 | Proc:7.0 | Disp:6.9 | Motion:0 | Drop:2
[720p] Cap:7.0 | Proc:6.9 | Disp:6.9 | Motion:3 | Drop:3
[720p] Cap:6.6 | Proc:6.7 | Disp:6.5 | Motion:16 | Drop:10
[720p] Cap:6.4 | Proc:6.5 | Disp:7.2 | Motion:0 | Drop:11
[720p] Cap:7.1 | Proc:7.1 | Disp:6.8 | Motion:11 | Drop:14

Stopping...
Cleanup...

FINAL STATISTICS (720p):
Capture FPS:  6.85
Process FPS:  6.86
Display FPS:  6.51
Dropped:      15 frames
Done
